In [1]:
# Install package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install Dijkstar 
import mysql.connector
from tqdm import tqdm
from dijkstar import Graph, find_path

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="915997582",
  database="EDDB"  
)

print(db)

cursor = db.cursor()

In [3]:
# How many systems does each faction control
query = """
SELECT COUNT(id), allegiance, sum(population) as populations
FROM system_populated
WHERE NOT allegiance = -1
GROUP BY allegiance
ORDER BY COUNT(id) DESC
"""

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

(10507, 'Independent')
(4556, 'Empire')
(4386, 'Federation')
(1092, 'Alliance')
(19, '-1')
(9, 'Pilots Federation')


In [4]:
# Find the commodity with the largest profit margin.
query = """
SELECT name, max_sell_price, min_buy_price
FROM commodity
ORDER BY (max_sell_price - min_buy_price) DESC
LIMIT 5
"""

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

('Tritium', 52455, 4005)
('Meta-Alloys', 236737, 190461)
('Agronomic Treatment', 30998, 322)
('Ceramic Composites', 27883, 77)
('CMM Composite', 20176, 4807)


In [5]:
# Find the system that sells a given commodity for the lowest price.

name = "Beer"

# get price
query = """
SELECT min_buy_price
FROM commodity
WHERE name = "{name}"
""".format(name=name)

cursor.execute(query)

results = cursor.fetchall()

price = 0

for x in results:
    price = x[0]

# where is it
query = """
SELECT system_populated.name
FROM listing
JOIN station ON station.id=listing.station_id
JOIN system_populated ON station.system_id=system_populated.id
JOIN commodity ON listing.commodity_id=commodity.id
WHERE commodity.name = "{name}" AND listing.buy_price = {price}
""".format(name=name, price=price)

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

('LFT 37',)


In [6]:
# Find the system that buys a given commodity for the highest price.
# BROKEN
name = "Beer"

# get price
query = """
SELECT max_sell_price
FROM commodity
WHERE name = "{name}"
""".format(name=name)

cursor.execute(query)

results = cursor.fetchall()

price = 0

for x in results:
    price = x[0]

# where is it
query = """
SELECT system_populated.name
FROM listing
JOIN station ON station.id=listing.station_id
JOIN system_populated ON station.system_id=system_populated.id
JOIN commodity ON listing.commodity_id=commodity.id
WHERE commodity.name = "{name}" AND listing.buy_price = {price}
""".format(name=name, price=price)

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

In [7]:
# Given a system, station, and range - find the best commodity to buy and sell locally.

trade_range = 12
system_name = "Nerthus"

# Get System Coords
query = """
SELECT x, y, z
FROM system_populated
WHERE name = '{name}'
""".format(name=system_name)

cursor.execute(query)

results = cursor.fetchall()

coords = results[0]

# Get all stations within range with listings
query = """
SELECT station.name, station.id, sys.name, sys.id, commodity.name, commodity.id, listing.sell_price, listing.buy_price, listing.supply
FROM system_populated as sys
JOIN station ON station.system_id = sys.id
JOIN listing ON station.id = listing.station_id
JOIN commodity ON commodity.id=listing.commodity_id
WHERE POWER({trade_range}, 2) >= (POWER(sys.x - {x}, 2) + POWER(sys.y - {y}, 2) + POWER(sys.z - {z}, 2))
""".format(trade_range=trade_range, x=coords[0], y=coords[1], z=coords[2])

cursor.execute(query)

results = cursor.fetchall()

stations = []

min_buy = dict()
max_sell = dict()
profit = dict()

for x in results:
    if x[8] > 0:
        if x[4] in min_buy:
            if min_buy[x[4]] > x[7]:
                min_buy[x[4]] = x[7]
        else:
            min_buy[x[4]] = x[7]

for x in results:
    if x[4] in min_buy:
        if x[4] in max_sell:
            if max_sell[x[4]] < x[6]:
                max_sell[x[4]] = x[6]
        else:
            max_sell[x[4]] = x[6]
            
for x in min_buy:
    profit[x] = max_sell[x] - min_buy[x]

max_profit_commodity = max(profit)
max_profit = profit[max(profit)]
buy_location = None
sell_location = None

for x in results:
    if x[6] == max_sell[max_profit_commodity]:
        sell_location = x
    if x[7] == min_buy[max_profit_commodity]:
        buy_location = x

print(f"Range: {trade_range}")
print(f"Center System: {system_name}")
print(f"Buy Location: {buy_location}")
print(f"Sell Location: {sell_location}")
print(f"Profit: {max_profit}")

Range: 12
Center System: Nerthus
Buy Location: ('Roberts Terminal', 13435, 'Smethells 119', 17035, 'Wine', 12, 376, 424, 9845)
Sell Location: ('Sterling Vista', 43976, 'LHS 3802', 12064, 'Wine', 12, 858, 0, 0)
Profit: 434


In [8]:
# Find the nearest system that provides a specified service also works for finding a black market because it is considered a service

current_system = "Nerthus"
service = "has_blackmarket"

# Get System Coords
query = """
SELECT x, y, z
FROM system_populated
WHERE name = '{name}'
""".format(name=current_system)

cursor.execute(query)

results = cursor.fetchall()

coords = results[0]

query = """
SELECT sys.name, station.name, station.allegiance, (POWER(sys.x - {x}, 2) + POWER(sys.y - {y}, 2) + POWER(sys.z - {z}, 2)) AS Distance
FROM station
JOIN system_populated AS sys
ON sys.id = station.system_id
WHERE {service} = 1
ORDER BY Distance ASC
LIMIT 5
""".format(service=service, x=coords[0], y=coords[1], z=coords[2])

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

('Two Ladies', 'Shinn Orbital', 'Independent', 54.40784219000002)
('Two Ladies', 'Beekman Platform', 'Independent', 54.40784219000002)
('Two Ladies', 'Julian Hub', 'Independent', 54.40784219000002)
('Two Ladies', 'Betancourt Horizons', 'Independent', 54.40784219000002)
('CD-46 14401', 'Klein Enterprise', 'Independent', 77.13722968999998)


In [9]:
# Find the rarest module and where to buy it

# Find rare module - that is, a module that is only sold at a single to a few locations
query="""
SELECT module_id, COUNT(*) AS count
FROM selling_module
GROUP BY module_id
ORDER BY count ASC
LIMIT 1
"""
cursor.execute(query)

results = cursor.fetchall()

module_id = results[0][0]

query="""
SELECT sys.name, station.name, module.ship, module.ed_symbol
FROM selling_module
JOIN station
ON selling_module.station_id = station.id
JOIN system_populated AS sys
ON station.system_id = sys.id
JOIN module
ON module.id = selling_module.module_id
WHERE selling_module.module_id = {module_id}
""".format(module_id=module_id)

cursor.execute(query)

results = cursor.fetchall()

print(results[0])

1761
('Kokojina', 'Ali Ring', '0', 'Hpt_Guardian_ShardCannon_Turret_Large')


In [10]:
# Find the best path between two systems given the max jump range of a ship

# yikes

In [11]:
# Buld an aggregation of listing data to display avg buy price, avg sell price, total supply, total demand, etc.
# We are assuming these values have not been aggregated already

#avg buy price
query = """
SELECT commodity.name, AVG(listing.buy_price) as avg
FROM listing
JOIN commodity
ON listing.commodity_id = commodity.id
WHERE listing.supply > 0
GROUP BY listing.commodity_id
ORDER BY avg ASC
"""

cursor.execute(query)

results = cursor.fetchall()

print(f"Avg buy aggregated with {len(results)} entries")

#avg sell price
query = """
SELECT commodity.name, AVG(listing.sell_price) as avg
FROM listing
JOIN commodity
ON listing.commodity_id = commodity.id
GROUP BY listing.commodity_id
ORDER BY avg ASC
"""

cursor.execute(query)

results = cursor.fetchall()

print(f"Avg sell aggregated with {len(results)} entries")

#galactic supply
query = """
SELECT commodity.name, SUM(listing.supply) as sup
FROM listing
JOIN commodity
ON listing.commodity_id = commodity.id
GROUP BY listing.commodity_id
ORDER BY sup DESC
"""

cursor.execute(query)

results = cursor.fetchall()

print(f"Galactic supply aggregated with {len(results)} entries")
for x in results:
    print(x)

#galactic demand
query = """
SELECT commodity.name, SUM(listing.demand) as dem
FROM listing
JOIN commodity
ON listing.commodity_id = commodity.id
GROUP BY listing.commodity_id
ORDER BY dem DESC
"""

cursor.execute(query)

results = cursor.fetchall()

print(f"Galactic demand aggregated with {len(results)} entries")
for x in results:
    print(x)

Avg buy aggregated with 197 entries
Avg sell aggregated with 201 entries
Galactic supply aggregated with 201 entries
('Hydrogen Fuel', Decimal('8622507996'))
('Food Cartridges', Decimal('7609744667'))
('Water Purifiers', Decimal('5593524505'))
('Crop Harvesters', Decimal('5405108424'))
('Aluminium', Decimal('4499168863'))
('Mineral Extractors', Decimal('4192065036'))
('Marine Equipment', Decimal('4067004415'))
('Copper', Decimal('3434573431'))
('Geological Equipment', Decimal('3260480238'))
('Polymers', Decimal('3223017381'))
('Titanium', Decimal('2829344848'))
('Scrap', Decimal('2746016414'))
('Fruit and Vegetables', Decimal('2651238771'))
('Cobalt', Decimal('2624364005'))
('Grain', Decimal('2591814512'))
('Synthetic Fabrics', Decimal('2377531050'))
('Building Fabricators', Decimal('2375073554'))
('Bauxite', Decimal('2130491662'))
('Algae', Decimal('2048706884'))
('Power Generators', Decimal('1862035931'))
('Lithium', Decimal('1647718923'))
('Atmospheric Processors', Decimal('16389715

In [21]:
# find the best path from point A to point B given a max jump range

# get the data

class System:
    def __init__(self, x, y, z, id, name):
        self.x = x
        self.y = y
        self.z = z
        self.name = name
        self.id = id

    def to_string(self):
        return f"{self.name},{self.x},{self.y},{self.z},{self.id}"

def get_system_by_id(id):
    query = """
    SELECT *
    FROM system_populated
    WHERE id = {id}
    """.format(id=id)
    cursor.execute(query)
    systems = cursor.fetchall()
    return systems[0]

nerthus_id = 14324
sol_id = 17072
ladies_id = 18097

query = """
SELECT *
FROM system_populated
"""
cursor.execute(query)

mysql_systems = cursor.fetchall()

systems = []

def mysql_to_class(mysql):
    return System(mysql[3], mysql[4], mysql[5], mysql[0], mysql[2])

for system in tqdm(mysql_systems, desc="Building system classes"):
    systems.append(mysql_to_class(system))

def economic_func(u, v, edge, prev_edge):
    return edge

def fastest_func(u, v, edge, prev_edge):
    return 1

def distance_between_systems(sys1, sys2):
    return ((sys1.x - sys2.x)**2 + (sys1.y - sys2.y)**2 + (sys1.z - sys2.z)**2)**0.5

# @param start : System to start path
# @param end : System to end path
# @param max_jump : 
# @param start System to start path
# @param start System to start path
# using System class
def trimmed_path(start, end, max_jump, buffer, systems):
    segment_length = distance_between_systems(start, end)
    max_sum_of_hypotenuse = buffer + segment_length
    graph = Graph()

    tot = 0

    for system_1 in tqdm(systems, desc=f"Building Graph buffer_length: {buffer}"):
        if max_sum_of_hypotenuse < distance_between_systems(system_1, start) + distance_between_systems(system_1, end):
            continue
        for system_2 in systems:
            if system_2.id == system_1.id:
                continue
            if max_sum_of_hypotenuse < distance_between_systems(system_2, start) + distance_between_systems(system_2, end):
                continue

            distance = distance_between_systems(system_1, system_2)
            if distance < max_jump:
                tot+=1
                graph.add_edge(system_1.id, system_2.id, distance)

    try:
        return True, find_path(graph, start.id, end.id, cost_func=economic_func)            
    except:
        return False, None, tot

start = mysql_to_class(get_system_by_id(nerthus_id))
end = mysql_to_class(get_system_by_id(sol_id))

buffer = 7
max_jump = 20

paths = trimmed_path(start, end, max_jump, buffer, systems)
for x in paths:
    print(x)

Building Graph buffer_length: 7: 100%|██████████| 20569/20569 [00:02<00:00, 7512.39it/s]True
PathInfo(nodes=[14324, 12064, 17784, 16629, 12066, 4592, 17072], edges=[10.132365453831598, 18.479799064113227, 9.328445660987684, 14.383080555378255, 10.756289439323394, 11.80240971158009], costs=[10.132365453831598, 18.479799064113227, 9.328445660987684, 14.383080555378255, 10.756289439323394, 11.80240971158009], total_cost=74.88238988521425)

